# Logical Battleship

This repository contains my solutions for the Artificial Intelligence (NLP) course that I attended during the second semester - academic year 2023/2024 - of M.Sc. in Machine Learning & Big Data at the University of Naples "Parthenope" (@uniparthenope). The course covered basic and introductive topics of Artificial Intelligence, and I completed the project, which is described below:

## The project

The purpose of the project is to create a knowledge agent capable of playing battleship and coding its knowledge base and rules using prepositional logic. 

## Assumption 

For simplicity's sake, it was decided to have a fixed arrangement of ships for the player and the agent, as the focus of the project is to develop the agent's reasoning ability and to provide him with an explicit knowledge representation. 
In particular for the latter aspect, the following were expressed through prepositional logic: the rules of the game, the structure of the ships (number of boxes for each type of ship), the number of ships available and the size of the playing field.

## Structure and instruments

Python was chosen as the programming language to realise the project, and in particular the Aima logic library was used to handle the agent logic in the form of prepositional logic. The programme consists of 3 files: the main, a file containing the implementation of the game and another containing the implementation of the agent. These are represented by the files respectively: logicalBattleship.py, battleshipGame.py and battleshipAgent.py .

In [1]:
# Installation
%pip install aima

You should consider upgrading via the '/Users/alfredomungari/Test/env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


### battleshipAgent.py

In this file, the BattleshipAgent class is defined, which defines the knowledge base (KB) of the agent, the extent of the KB, the queries made on the KB, the moves made by the agent and its inference mechanism. Class attributes define the size of the battlefield, the knowledge base, the ships hit and miss. 

### knowledge_base_definition()
The knowledge_base_definition() method defines a priori knowledge, some strategies for performing hits by inference and some notions. Three symbols are defined: H, M and S, respectively representing a Hit (a square of a ship one hit), a Miss (an empty hit), Sunk (a square of a ship one hit and sunk). Different cases are handled as background knowledge, the first is the case where case where there are two adjacent hits, on the flanks of the two hits will be two misses, case when there are one hit and three adjacent misses (edges of the grid are excluded), case when there are one hit and two adjacent misses, on the left and right edge of the grid (corners of the grid are excluded),  case when there are one hit and two adjacent misses on the left and right edge of the grid (corners of the grid are excluded), case when there are one hit and two adjacent misses on the lowest and highest edge of the grid (corners of the grid are excluded), case which involve corners of the grid (G[0][0], G[0][N], G[M][0] and G[M][N]) and cases where there are four misses at the four sides and an empty cell in the centre in the central cell there is a miss to infer.

### add_knowledge()
Extends the KB, in particular uncovered cells such as Hits, Misses and Sunks are inserted.

### get_adjacent_cells()
Given a cell returns the four adjacent cells.

### check_globally()
This method represents a possible move by the agent, it is called every 10 moves by the agent, once the corresponding turn has arrived, the agent does not attempt to make a hit based on the most recent previous hits, instead it scans the entire grid, cell by cell, by row, trying to locate all the Miss cells and adding them to its knowledge base, but without a hit being attempted yet. If the agent infers the presence of a Hit, based on its KB, it returns the cell corresponding to the Hit as a move, if no Hit is found an empty move is returned and the agent makes a move by choosing it via the choose_the_next_move() method.

### choose_the_next_move()
This method represents the agent's reasoning, in particular the move to be made is defined. If a hit has been made in recent previous moves, an attempt is made to hit around it otherwise a random pair of coordinates is generated and a random hit is made.

In [ ]:
from aima.logic import PropDefiniteKB, expr, pl_fc_entails
import random

class BattleshipAgent:
    def __init__(self, size):
        self.size = size  # Grid size
        self.kb = PropDefiniteKB()  # Knowledge base
        self.hits = []  # List of successfully hit boxes
        self.misses = []  # List of boxes hit unsuccessfully
        self.knowledge_base_definition()

    def knowledge_base_definition(self):
        '''
        Defines the agent's knowledge base by codifying game rules and strategies
        '''
        # H = Hit  M = Miss  S = Sunk  
        #                                                                                                          M M    M H M
        # Handle case where there are two adjacent hits, on the flanks of the two hits will be two misses (e.g.:   H H    M H M )
        #                                                                                                          M M  
                                                                                          
        for x in range(self.size):
            for y in range(self.size):
                if 0 < x < self.size and 0 < y < self.size:
                    # Handles a miss in the upper left corner in case of two adjacent horizontal misses
                    self.kb.tell(expr(f"(Hit_({x + 1}, {y}) & Hit_({x + 1}, {y + 1}))==>Miss_({x}, {y})"))
                    # Handles a miss in the upper right corner in case of two adjacent horizontal misses
                    self.kb.tell(expr(f"(Hit_({x + 1}, {y}) & Hit_({x + 1}, {y - 1}))==>Miss_({x}, {y})"))
                    # Handles a miss in the lower right corner in case of two adjacent horizontal misses
                    self.kb.tell(expr(f"(Hit_({x - 1}, {y}) & Hit_({x - 1}, {y - 1}))==>Miss_({x}, {y})"))
                    # Handles a miss in the lower left corner in case of two adjacent horizontal misses
                    self.kb.tell(expr(f"(Hit_({x - 1}, {y}) & Hit_({x - 1}, {y + 1}))==>Miss_({x}, {y})"))
                    # Handles a miss in the upper left corner in case of two adjacent vertical misses
                    self.kb.tell(expr(f"(Hit_({x}, {y + 1}) & Hit_({x + 1}, {y + 1}))==>Miss_({x}, {y})"))
                    # Handles a miss in the upper right corner in case of two adjacent vertical misses
                    self.kb.tell(expr(f"(Hit_({x}, {y - 1}) & Hit_({x + 1}, {y - 1}))==>Miss_({x}, {y})"))
                    # Handles a miss in the lower right corner in case of two adjacent vertical misses
                    self.kb.tell(expr(f"(Hit_({x}, {y - 1}) & Hit_({x - 1}, {y - 1}))==>Miss_({x}, {y})"))
                    # Handles a miss in the lower left corner in case of two adjacent vertical misses
                    self.kb.tell(expr(f"(Hit_({x}, {y + 1}) & Hit_({x - 1}, {y + 1}))==>Miss_({x}, {y})"))

        # Handle previous case in the corners of the grid
        self.kb.tell(expr(f"(Hit_(0, 0) & Hit_(0, 1))==>Miss_(1, 0)"))
        self.kb.tell(expr(f"(Hit_(0, 0) & Hit_(1, 0))==>Miss_(0, 1)"))
        self.kb.tell(expr(f"(Hit_(0, 9) & Hit_(0, 8))==>Miss_(1, 9)"))
        self.kb.tell(expr(f"(Hit_(0, 9) & Hit_(1, 9))==>Miss_(0, 8)"))
        self.kb.tell(expr(f"(Hit_(9, 0) & Hit_(9, 1))==>Miss_(8, 0)"))
        self.kb.tell(expr(f"(Hit_(9, 0) & Hit_(8, 0))==>Miss_(9, 1)"))
        self.kb.tell(expr(f"(Hit_(9, 9) & Hit_(9, 8))==>Miss_(8, 9)"))
        self.kb.tell(expr(f"(Hit_(9, 9) & Hit_(8, 9))==>Miss_(9, 8)"))

        # Case when there are one hit and three adjacent misses (edges of the grid are excluded)
        for x in range(1, self.size - 1):
            for y in range(1, self.size - 1):
                self.kb.tell(expr(f"(Hit_({x}, {y}) & Miss_({x}, {y - 1}) & Miss_({x - 1}, {y}) & Miss_({x + 1}, {y}))==>Hit_({x}, {y + 1})"))
                self.kb.tell(expr(f"(Hit_({x}, {y}) & Miss_({x}, {y + 1}) & Miss_({x + 1}, {y}) & Miss_({x - 1}, {y}))==>Hit_({x}, {y - 1})"))
                self.kb.tell(expr(f"(Hit_({x}, {y}) & Miss_({x - 1}, {y}) & Miss_({x}, {y - 1}) & Miss_({x}, {y + 1}))==>Hit_({x + 1}, {y})"))
                self.kb.tell(expr(f"(Hit_({x}, {y}) & Miss_({x + 1}, {y}) & Miss_({x}, {y - 1}) & Miss_({x}, {y + 1}))==>Hit_({x - 1}, {y})"))

        # Case when there are one hit and two adjacent misses, on the left and right edge of the grid (corners of the grid are excluded)
        for x in range(1, self.size - 1):
            self.kb.tell(expr(f"(Hit_({x}, 0) & Miss_({x - 1}, 0) & Miss_({x + 1}, 0))==>Hit_({x}, 1)"))
            self.kb.tell(expr(f"(Hit_({x}, 0) & Miss_({x}, 1) & Miss_({x + 1}, 0))==>Hit_({x - 1}, 0)"))
            self.kb.tell(expr(f"(Hit_({x}, 0) & Miss_({x}, 1) & Miss_({x - 1}, 0))==>Hit_({x + 1}, 0)"))
            self.kb.tell(expr(f"(Hit_({x}, 9) & Miss_({x - 1}, 9) & Miss_({x + 1}, 9))==>Hit_({x}, 8)"))
            self.kb.tell(expr(f"(Hit_({x}, 9) & Miss_({x}, 8) & Miss_({x + 1}, 9))==>Hit_({x - 1}, 9)"))
            self.kb.tell(expr(f"(Hit_({x}, 9) & Miss_({x}, 8) & Miss_({x - 1}, 9))==>Hit_({x + 1}, 9)"))

        # Case when there are one hit and two adjacent misses, on the lowest and highest edge of the grid (corners of the grid are excluded)
        for y in range(1, self.size - 1):
            self.kb.tell(expr(f"(Hit_(0, {y}) & Miss_(0, {y - 1}) & Miss_(0, {y + 1}))==>Hit_(1, {y})"))
            self.kb.tell(expr(f"(Hit_(0, {y}) & Miss_(1, {y}) & Miss_(0, {y + 1}))==>Hit_(1, {y - 1})"))
            self.kb.tell(expr(f"(Hit_(0, {y}) & Miss_(1, {y}) & Miss_(0, {y - 1}))==>Hit_(1, {y + 1})"))
            self.kb.tell(expr(f"(Hit_(9, {y}) & Miss_(9, {y - 1}) & Miss_(9, {y + 1}))==>Hit_(1, {y})"))
            self.kb.tell(expr(f"(Hit_(9, {y}) & Miss_(8, {y}) & Miss_(9, {y + 1}))==>Hit_(9, {y - 1})"))
            self.kb.tell(expr(f"(Hit_(9, {y}) & Miss_(8, {y}) & Miss_(9, {y - 1}))==>Hit_(9, {y + 1})"))

        # Case which involve corners of the grid (G[0][0], G[0][N], G[M][0] and G[M][N])
        self.kb.tell(expr(f"(Hit_(0, 0) & Miss_(0, 1))==>Hit_(1, 0)"))
        self.kb.tell(expr(f"(Hit_(0, 0) & Miss_(1, 0))==>Hit_(0, 1)"))
        self.kb.tell(expr(f"(Hit_(9, 0) & Miss_(8, 0))==>Hit_(9, 1)"))
        self.kb.tell(expr(f"(Hit_(9, 0) & Miss_(9, 1))==>Hit_(8, 0)"))
        self.kb.tell(expr(f"(Hit_(0, 9) & Miss_(0, 8))==>Hit_(1, 9)"))
        self.kb.tell(expr(f"(Hit_(0, 9) & Miss_(1, 9))==>Hit_(0, 8)"))
        self.kb.tell(expr(f"(Hit_(9, 9) & Miss_(8, 9))==>Hit_(9, 8)"))
        self.kb.tell(expr(f"(Hit_(9, 9) & Miss_(9, 8))==>Hit_(8, 9)"))

        #                    |M|
        # Hendles cases as: M| |M in the central cell there is a miss to infer
        #                    |M|
        for x in range(self.size):
            for y in range(self.size):
                if 0 < x < self.size and 0 < y < self.size and 0 < x < self.size and 0 < y < self.size:
                    self.kb.tell(expr(f"(Miss_({x - 1}, {y}) & Miss_({x + 1}, {y}) & Miss_({x}, {y - 1}) & Miss_({x}, {y + 1}))==>Miss_({x}, {y})"))

    def add_knowledge(self, cell, result):
        """Adds knowledge to KB based on stroke result"""
        if result == "hit":
            self.hits.append(cell)
            self.kb.tell(expr(f"Hit_{cell}"))
        elif result == "miss":
            self.misses.append(cell)
            self.kb.tell(expr(f"Miss_{cell}"))
        else:
            self.kb.tell(expr(f"Sunk_{cell}"))

    def get_adjacent_cells(self, cell):
        """Returns the boxes adjacent to a given cell"""
        x, y = cell
        adjacent = [(x + dx, y + dy) for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]]
        return [(i, j) for i, j in adjacent if 0 <= i < self.size and 0 <= j < self.size]
    
    def check_globally(self):
        for x in range(self.size):
            for y in range(self.size):
                cell = (x, y)
                if cell not in self.hits and cell not in self.misses:
                    if pl_fc_entails(self.kb, expr(f"Hit_{cell}")):
                        print(f"\033[32mInferred ({chr(65 + x)}, {y}) as hit\033[0m")
                        return cell
                    if pl_fc_entails(self.kb, expr(f"Miss_{cell}")):
                        print(f"\033[32mInferred ({chr(65 + x)}, {y}) as miss\033[0m")
                        self.add_knowledge(cell, "miss")
        return()

    def choose_next_move(self):
        """Chooses the next move based on current knowledge base"""
        for cell in self.hits:
            for adj in self.get_adjacent_cells(cell):
                if adj not in self.hits and adj not in self.misses:
                    if pl_fc_entails(self.kb, expr(f"Miss_{adj}")):
                        print(f"\033[32mInferred ({chr(65 + adj[0])}, {adj[1]}) as miss\033[0m")
                        self.add_knowledge(adj, "miss")
                    else:
                        return adj
                
        # If there are no recent hits, choose a random cell
        while True:
            move = (random.randint(0, self.size-1), random.randint(0, self.size-1))
            if move not in self.hits and move not in self.misses:
                return move

### battleshipGame.py
This file defines the implementation of the game, agent turns and human player turns are managed, and various game situations, all this represented by the BattleshipGame class. Class attributes represent: the size of the playing field, the disappointing ships and the agent, the sunk ships by the user and the agent, the agent itself, the grids containing the user’s and the agent’s ships, the grids containing the hits made by the user and the agent, the Hits and Miss of the user.

### create_grid()
This method creates a grid of given size, representing a playing field for the naval battle.

### prin_grid()
Method to show a video of a playing field.

### update_grid()
Updates the grid representing the playing field, depending on whether the hit made in the given turn is a Hit or a Miss.

### check_hit()
Check if the shot you choose is a Hit or a Miss

### update_sunk_adjiances()
Upgrade miss cells around sunk cell

### check_sunk()
Check after a hit if the hit ship was also sunk

### get_letter()
Allows the user to choose a letter between A and J

### get_number()
Allows the user to choose a number between 0 and 9

### human_next_move()
Make the user’s move

### play()
Simulate the game

In [ ]:
from battleshipAgent import *

class BattleshipGame:
    def __init__(self, size, human_ships, agent_ships):
        self.size = size  # Grid size
        self.human_ships = human_ships  # List of human ships, each represented by a list of tuples (cell positions occupied by the ship)
        self.agent_ships = agent_ships  # List of agent ships, each represented by a list of tuples (cell positions occupied by the ship)
        self.human_sunk = [] # List of user's sunk boxes
        self.agent_sunk = [] # List of agent's sunk boxes
        self.agent = BattleshipAgent(size) # Agent instance
        self.human_grid_own = self.create_grid() # User grid where one's pawns are placed
        self.human_grid_adv = self.create_grid() # User grid where the moves made are memorized
        self.agent_grid_own = self.create_grid() # Agent grid where one's pawns are placed
        self.agent_grid_adv = self.create_grid() # Agent grid where the moves made are memorized
        self.human_hits = [] # List of boxes hit successfully by user
        self.human_misses = [] # List of boxes hit unsuccessfully by user

    def create_grid(self):
        grid = []
        for i in range(self.size):
            grid.append([])
            for _ in range(self.size):
                grid[i].append('*')
        return grid

    def print_grid(self, grid, turn):
        if turn == 'A':
            print("\033[32m   \033[0m" + "\033[32m \033[0m".join(f"\033[32m{i:2}\033[0m" for i in range(0, self.size)))
        else:
            print("   " + " ".join(f"{i:2}" for i in range(0, self.size)))
        for i in range(self.size):
            row_label = chr(65 + i)
            row_content = " ".join(f" {grid[i][j]}" for j in range(len(grid[0])))
            if turn == 'A':
                print(f"\033[32m{row_label}  {row_content}\033[0m")
            else:
                print(f"{row_label}  {row_content}")
        print('\n')

    def update_grid(self, turn):
        if turn == 'A':
            for cell in self.agent.hits:
                if self.agent_grid_adv[cell[0]][cell[1]] != 'S':
                    self.agent_grid_adv[cell[0]][cell[1]] = 'H'
            for cell in self.agent.misses:
                self.agent_grid_adv[cell[0]][cell[1]] = 'M'
        else:
            for cell in self.human_hits:
                self.human_grid_adv[cell[0]][cell[1]] = 'H'
            for cell in self.human_misses:
                self.human_grid_adv[cell[0]][cell[1]] = 'M'

    def check_hit(self, cell, ships):
        """Check if a cell hits a ship"""
        for ship in ships:
            if cell in ship:
                return "hit"
        return "miss"

    def update_sunk_adjiances(self, ship):
        '''Upgrade miss cells around sunk cells'''
        for cell in ship:
            self.agent.add_knowledge(cell, "sunk")
            self.agent_grid_adv[cell[0]][cell[1]] = 'S'
            adj = self.agent.get_adjacent_cells(cell)
            for cell in adj:
                if cell not in self.agent.misses and cell not in self.agent.hits:
                    self.agent.add_knowledge(cell, "miss")
                    x, y = cell
                    self.agent_grid_adv[x][y] = 'M'

    def check_sunk(self, cell, turn):
        '''Check after a hit if the hit ship was also sunk'''
        if turn == 'A':
            for ship in self.human_ships:
                if ship not in self.human_sunk and all(cell in self.agent.hits for cell in ship):
                    self.human_sunk.append(ship)
                    print("\033[32msunk!\033[0m\n")
                    self.update_sunk_adjiances(ship)
        else:
            for ship in self.agent_ships:
                if ship not in self.agent_sunk and all(cell in self.human_hits for cell in ship):
                    self.agent_sunk.append(ship)
                    print("Sunk!\n")
                    for cell in ship:
                        self.human_grid_adv[cell[0]][cell[1]] = 'S'

    def get_letter(self):
        while True:
            letter = input("Choose a letter between A and J: ").upper()
            if letter in 'ABCDEFGHIJ':
                return letter
            else:
                print("Invalid letter. Try again.")

    def get_number(self):
        while True:
            number = int(input("Choose a number between 0 and 9: "))
            if 0 <= number <= 9:
                return number
            else:
                print("Invalid number. Try again.")

    def human_next_move(self):
        x = self.get_letter()
        y = self.get_number()
        return (ord(x) - 65, y)

    def play(self):
        """Simulate the game"""
        for ship in self.human_ships:
            for i, j in ship:
                self.human_grid_own[i][j] = 'P'
        for ship in self.agent_ships:
            for i, j in ship:
                self.agent_grid_own[i][j] = 'P'

        turn = random.choice(['H', 'A'])
        human_move_counter = agent_move_counter = 0

        while True:
            move = ()
            if (turn == 'A'):
                agent_move_counter += 1
                print(f"\033[32mAI turn: {agent_move_counter}\033[0m\n")
                if agent_move_counter % 10 == 0:
                    move = self.agent.check_globally()
                if move == ():
                    move = self.agent.choose_next_move()
                print(f"\033[32mAI tries: ({chr(65 + move[0])}, {move[1]})\033[0m")
                result = self.check_hit(move, self.human_ships)
                print(f"\033[32m{result}!\033[0m\n")
                self.agent.add_knowledge(move, result)
                self.update_grid(turn)
                self.check_sunk(move, turn)
                print("\033[32mUser own grid:\033[0m\n")
                self.print_grid(self.human_grid_own, turn)
                print("\033[32mAI opponent grid:\033[0m\n")
                self.print_grid(self.agent_grid_adv, turn)

                if len(self.agent.hits) == sum(len(ship) for ship in self.human_ships):
                    print("\033[32mAll ships sunk!\033[0m")
                    print("\033[32mAI won!\033[0m\n")
                    break
                turn = 'H'
            else:
                human_move_counter += 1
                print(f"AI turn: {human_move_counter}\n")
                move = self.human_next_move()
                print(f"User tries: ({chr(65 + move[0])}, {move[1]})")
                result = self.check_hit(move, self.agent_ships)
                if result == 'hit':
                    self.human_hits.append(move)
                else:
                    self.human_misses.append(move)
                print(f"{result}!\n")
                self.update_grid(turn)
                self.check_sunk(move, turn)
                print("AI own grid:\n")
                self.print_grid(self.agent_grid_own, turn)
                print("User opponent grid:\n")
                self.print_grid(self.human_grid_adv, turn)

                if len(self.human_hits) == sum(len(ship) for ship in self.agent_ships):
                    print("All ships sunk!")
                    print("Human won!")
                    break
                turn = 'A'

## logicalBattleship.py

In this file, the player's and agent's ships are laid out and the BattleshipGame class is called, which takes the ship arrangements as input. Finally, the game is started via the play() method of the BattleshipGame class.

In [ ]:
from battleshipGame import *

def main():
    A, B, C, D, E, F, G, H, I, J = range(10)
    grid_size = 10
    human_ships = [
        # 4 PatrolBoat (2 cells) 
        [(B, 8), (C, 8)], 
        [(F, 8), (F, 9)],
        [(J, 4), (J, 5)],
        [(I, 9), (J, 9)],
        # 3 Destroyer (3 cells)
        [(H, 6), (H, 7), (H, 8)],
        [(C, 2), (D, 2), (E, 2)],
        [(J, 0), (J, 1), (J, 2)],
        # 2 Battleship (4 cells)
        [(C, 0), (D, 0), (E, 0), (F, 0)],
        [(E, 4), (E, 5), (E, 6), (E, 7)],
        # 1 Carrier (6 cells)
        [(A, 0), (A, 1), (A, 2), (A, 3), (A, 4), (A, 5)]
    ]
    agent_ships = [
        # 4 PatrolBoat (2 cells) 
        [(A, 0), (A, 1)], 
        [(C, 2), (C, 3)],
        [(E, 4), (E, 5)],
        [(I, 1), (J, 1)],
        # 3 Destroyer (3 cells)
        [(A, 7), (B, 7), (C, 7)],
        [(I, 3), (I, 4), (I, 5)],
        [(E, 0), (E, 1), (E, 2)],
        # 2 Battleship (4 cells)
        [(G, 2), (G, 3), (G, 4), (G, 5)],
        [(G, 7), (H, 7), (I, 7), (J, 7)],
        # 1 Carrier (6 cells)
        [(D, 9), (E, 9), (F, 9), (G, 9), (H, 9), (I, 9)]
    ]
    game = BattleshipGame(grid_size, human_ships, agent_ships)
    game.play()

if __name__ == "__main__":
    main()